In [1]:
from __future__ import division
from __future__ import unicode_literals
from __future__ import absolute_import
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
#from rdkit.Avalon.pyAvalonTools import GetAvalonFP
import rdkit.rdBase
from rdkit.Chem.MACCSkeys import GenMACCSKeys
from rdkit import DataStructs
from rdkit.DataStructs import BitVectToText
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.Chem import Descriptors as Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd 
from rdkit.Chem import PandasTools as PandasTools
from rdkit.Chem import Descriptors as Descriptors

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from rdkit.Chem import DataStructs
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


In [3]:
model_tox_data = pd.read_csv('../data/Alloriginaldata_19911.csv') 
model_tox_data

,smiles_r,TOXICITY,Source_rank,Source,Data
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI
...,...,...,...,...,...
19906,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox
19907,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19908,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19909,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0,16,Reactive Metabolite Mazzolari et al,LivTox


In [4]:
train_data_tox = model_tox_data[model_tox_data["Data"]!="DILI"]
train_data_tox = train_data_tox[["smiles_r","TOXICITY"]].reset_index(drop=True)
train_data_tox

,smiles_r,TOXICITY
0,C[C@H](N)C(=O)N[C@@H](C)C(=O)NC1[C@@H]2CN(c3nc...,1
1,CN(C)CCCC1c2ccccc2Nc2ccc(Cl)cc21,1
2,NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1,1
3,O=c1[nH]c(=O)n([C@@H]2O[C@H](CO)[C@@H](O)[C@@H...,1
4,CN1CCCC(CN2c3ccccc3Sc3ccccc32)C1,1
...,...,...
18631,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0
18632,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0
18633,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0
18634,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0


In [5]:
outer_test_data_tox = model_tox_data[model_tox_data["Data"]=="DILI"]
outer_test_data_tox=outer_test_data_tox[["smiles_r","TOXICITY"]].reset_index(drop=True)
outer_test_data_tox

,smiles_r,TOXICITY
0,CCN(CC)c1cc(C)nc2ncnn12,1
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1
...,...,...
1270,O=C(N[C@@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)...,0
1271,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0
1272,C=C1/C(=C\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H...,0
1273,CCC(=O)NCC[C@@H]1CCc2ccc3c(c21)CCO3,0


In [6]:
def MorganFingerprint(s):
    x = Chem.MolFromSmiles(s)
    return (AllChem.GetMorganFingerprintAsBitVect(x,2,2048))

In [7]:
MorganFingerprint_array = np.stack(model_tox_data['smiles_r'].apply(MorganFingerprint))

Morgan_fingerprint_collection = []
for x in np.arange(MorganFingerprint_array.shape[1]):
    x = "Mfp"+str(x)
    Morgan_fingerprint_collection.append(x)

Morgan_fingerprint_table = pd.DataFrame(MorganFingerprint_array, columns=Morgan_fingerprint_collection)

Morgan_fingerprint_table

[23:42:14] Unusual charge on atom 16 number of radical electrons set to zero
[23:42:15] WARNING: not removing hydrogen atom without neighbors
[23:42:15] WARNING: not removing hydrogen atom without neighbors
[23:42:15] WARNING: not removing hydrogen atom without neighbors
[23:42:15] WARNING: not removing hydrogen atom without neighbors
[23:42:15] WARNING: not removing hydrogen atom without neighbors
[23:42:15] WARNING: not removing hydrogen atom without neighbors


,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,Mfp5,Mfp6,Mfp7,Mfp8,Mfp9,...,Mfp2038,Mfp2039,Mfp2040,Mfp2041,Mfp2042,Mfp2043,Mfp2044,Mfp2045,Mfp2046,Mfp2047
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19908,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19909,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [8]:
def MACCSKeysFingerprint(s):
    x = Chem.MolFromSmiles(s)
    return (AllChem.GetMACCSKeysFingerprint(x))

MACCSfingerprint_array = np.stack(model_tox_data['smiles_r'].apply(MACCSKeysFingerprint))

MACCS_collection = []
for x in np.arange(MACCSfingerprint_array.shape[1]):
    x = "MACCS"+str(x)
    MACCS_collection.append(x)

MACCSfingerprint_table = pd.DataFrame(MACCSfingerprint_array, columns=MACCS_collection)

MACCSfingerprint_table

[23:42:42] Unusual charge on atom 16 number of radical electrons set to zero
[23:42:48] WARNING: not removing hydrogen atom without neighbors
[23:42:48] WARNING: not removing hydrogen atom without neighbors
[23:42:49] WARNING: not removing hydrogen atom without neighbors
[23:42:49] WARNING: not removing hydrogen atom without neighbors
[23:42:49] WARNING: not removing hydrogen atom without neighbors
[23:42:49] WARNING: not removing hydrogen atom without neighbors


,MACCS0,MACCS1,MACCS2,MACCS3,MACCS4,MACCS5,MACCS6,MACCS7,MACCS8,MACCS9,...,MACCS157,MACCS158,MACCS159,MACCS160,MACCS161,MACCS162,MACCS163,MACCS164,MACCS165,MACCS166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
19907,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0
19908,0,0,0,0,0,0,0,0,1,0,...,1,1,1,1,1,1,1,1,1,0
19909,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0


In [9]:
2048+166

2214

In [10]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
from rdkit import Chem
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

print(len(calc.descriptors))

Ser_Mol = model_tox_data['smiles_r'].apply(Chem.MolFromSmiles)

# as pandas
Mordred_table=  calc.pandas(Ser_Mol)
Mordred_table = Mordred_table.astype('float')
#Mordred_table['smiles_r'] = model_tox_data['smiles_r']
Mordred_table

1613


[23:42:55] Unusual charge on atom 16 number of radical electrons set to zero
[23:42:56] WARNING: not removing hydrogen atom without neighbors
[23:42:57] WARNING: not removing hydrogen atom without neighbors
[23:42:57] WARNING: not removing hydrogen atom without neighbors
[23:42:57] WARNING: not removing hydrogen atom without neighbors
[23:42:57] WARNING: not removing hydrogen atom without neighbors
[23:42:57] WARNING: not removing hydrogen atom without neighbors
  0%|                                        | 31/19911 [00:01<11:20, 29.20it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  1%|▏                                     | 114/19911 [00:01<02:32, 130.06it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 63%|███████████████████████▍             | 12581/19911 [02:58<02:52, 42.43it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5


 97%|██████████████████████████████████▉ | 19351/19911 [04:16<00:05, 104.68it/s][23:47:14] WARNING: not removing hydrogen atom without neighbors
[23:47:14] WARNING: not removing hydrogen atom without neighbors
[23:47:14] WARNING: not removing hydrogen atom without neighbors
 98%|████████████████████████████████████▏| 19485/19911 [04:18<00:04, 96.17it/s][23:47:16] WARNING: not removing hydrogen atom without neighbors
[23:47:16] WARNING: not removing hydrogen atom without neighbors
[23:47:16] WARNING: not removing hydrogen atom without neighbors
 98%|███████████████████████████████████▍| 19583/19911 [04:18<00:02, 158.89it/s][23:47:16] WARNING: not removing hydrogen atom without neighbors
[23:47:16] WARNING: not removing hydrogen atom without neighbors
[23:47:16] WARNING: not removing hydrogen atom without neighbors
 99%|████████████████████████████████████▌| 19646/19911 [04:20<00:04, 59.71it/s][23:47:17] WARNING: not removing hydrogen atom without neighbors
[23:47:17] WARNING: not removi

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,11.301778,10.744632,0.0,0.0,19.043901,2.439076,4.810541,19.043901,1.269593,3.635979,...,9.602585,60.727536,205.132745,6.837758,338.0,22.0,76.0,90.0,5.305556,3.500000
1,13.763198,11.933749,0.0,1.0,23.181656,2.267318,4.534637,23.181656,1.220087,3.815888,...,9.339525,51.036465,264.147393,6.773010,844.0,23.0,86.0,93.0,7.805556,4.500000
2,19.189342,15.085744,0.0,1.0,32.341901,2.412835,4.825670,32.341901,1.293676,4.140144,...,10.080168,59.651481,355.145140,7.556280,1580.0,39.0,128.0,149.0,8.138889,5.666667
3,16.401355,13.440483,0.0,1.0,27.774822,2.397603,4.795207,27.774822,1.262492,3.989231,...,9.773777,55.480289,319.181526,6.649615,1212.0,31.0,106.0,120.0,7.666667,5.138889
4,20.591625,16.954561,0.0,1.0,34.701148,2.478045,4.904354,34.701148,1.285228,4.213576,...,10.168118,70.135184,387.151368,7.743027,1747.0,48.0,138.0,163.0,9.250000,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,19.443821,17.057231,0.0,0.0,31.485243,2.497496,4.989296,31.485243,1.259410,4.148861,...,10.241209,73.142299,349.115045,8.951668,1359.0,41.0,132.0,157.0,8.840278,5.458333
19907,24.981955,21.020184,0.0,0.0,39.278775,2.710620,5.371735,39.278775,1.267057,4.407257,...,10.959036,84.585213,428.147118,7.784493,2102.0,69.0,184.0,237.0,11.708333,6.513889
19908,25.650763,20.327383,0.0,3.0,42.760202,2.447383,4.894766,42.760202,1.257653,4.433319,...,10.360058,70.097297,473.263819,6.858896,4012.0,50.0,168.0,194.0,11.611111,7.833333
19909,37.521401,30.002431,0.0,0.0,61.187394,2.843212,5.496701,61.187394,1.301859,4.830687,...,11.424389,108.553825,648.293448,7.124104,6986.0,109.0,278.0,362.0,16.645833,10.138889


In [23]:
Mordred_table = Mordred_table.dropna(axis='columns')
Mordred_table

,ABC,ABCGG,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,11.301778,10.744632,0.0,0.0,9.0,10.0,30.0,15.0,0.0,0.0,...,6.605298,9.602585,60.727536,205.132745,6.837758,338.0,22.0,76.0,90.0,3.500000
1,13.763198,11.933749,0.0,1.0,6.0,6.0,39.0,19.0,0.0,0.0,...,0.000000,9.339525,51.036465,264.147393,6.773010,844.0,23.0,86.0,93.0,4.500000
2,19.189342,15.085744,0.0,1.0,16.0,17.0,47.0,25.0,0.0,0.0,...,0.000000,10.080168,59.651481,355.145140,7.556280,1580.0,39.0,128.0,149.0,5.666667
3,16.401355,13.440483,0.0,1.0,10.0,11.0,48.0,22.0,0.0,0.0,...,0.000000,9.773777,55.480289,319.181526,6.649615,1212.0,31.0,106.0,120.0,5.138889
4,20.591625,16.954561,0.0,1.0,12.0,12.0,50.0,27.0,0.0,0.0,...,5.379897,10.168118,70.135184,387.151368,7.743027,1747.0,48.0,138.0,163.0,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,19.443821,17.057231,0.0,0.0,17.0,17.0,39.0,25.0,0.0,0.0,...,6.259581,10.241209,73.142299,349.115045,8.951668,1359.0,41.0,132.0,157.0,5.458333
19907,24.981955,21.020184,0.0,0.0,5.0,5.0,55.0,31.0,0.0,0.0,...,7.669962,10.959036,84.585213,428.147118,7.784493,2102.0,69.0,184.0,237.0,6.513889
19908,25.650763,20.327383,0.0,3.0,6.0,6.0,69.0,34.0,0.0,0.0,...,0.000000,10.360058,70.097297,473.263819,6.858896,4012.0,50.0,168.0,194.0,7.833333
19909,37.521401,30.002431,0.0,0.0,6.0,6.0,91.0,47.0,0.0,3.0,...,8.945724,11.424389,108.553825,648.293448,7.124104,6986.0,109.0,278.0,362.0,10.138889


In [24]:
def get_num_charged_atoms_neg(mol):
    mol_h = Chem.AddHs(mol)
    Chem.rdPartialCharges.ComputeGasteigerCharges(mol_h)
     
    positive = 0
    negative = 0
     
    for atom in mol_h.GetAtoms():
        if float(atom.GetProp('_GasteigerCharge')) <= 0:
            negative += 1
     
    return negative

def get_num_charged_atoms_pos(mol):
    mol_h = Chem.AddHs(mol)
    Chem.rdPartialCharges.ComputeGasteigerCharges(mol_h)
     
    positive = 0
    negative = 0
     
    for atom in mol_h.GetAtoms():
        if float(atom.GetProp('_GasteigerCharge')) >= 0:
            positive += 1
    return positive

def get_assembled_ring(mol):
    ring_info = mol.GetRingInfo()
    num_ring = ring_info.NumRings()
    ring_atoms = ring_info.AtomRings()
    num_assembled = 0
     
    for i in range(num_ring):
        for j in range(i+1, num_ring):
            x = set(ring_atoms[i])
            y = set(ring_atoms[j])
            if not x.intersection(y): # 2つの環が縮環でない場合に
                for x_id in x:
                    x_atom = mol.GetAtomWithIdx(x_id)
                    neighbors = [k.GetIdx() for k in x_atom.GetNeighbors()]
                    for x_n in neighbors:
                        if x_n in y: # 環同士を繋ぐ結合があるか否か
                            num_assembled += 1
     
    return num_assembled

def get_num_stereocenters(mol):
    return AllChem.CalcNumAtomStereoCenters(mol) + AllChem.CalcNumUnspecifiedAtomStereoCenters(mol)

def calc_descriptors(dataframe):
    mols = dataframe.smiles_r.apply(Chem.MolFromSmiles)
    descr = []
    for m in mols:
        descr.append([ Descriptors.TPSA(m),
               Descriptors.NumRotatableBonds(m),
               AllChem.CalcNumRings(m),
               Descriptors.NumAromaticRings(m),
               Descriptors.NumHAcceptors(m),
               Descriptors.NumHDonors(m),
               Descriptors.FractionCSP3(m),
               Descriptors.MolLogP(m) ,
               Descriptors.NHOHCount(m),
               Descriptors.NOCount(m),
               Descriptors.NumHeteroatoms(m),
               get_num_charged_atoms_pos(m),
               get_num_charged_atoms_neg(m),
               get_assembled_ring(m),
               get_num_stereocenters(m)])
    descr = np.asarray(descr)
    return(descr)

descs = [ 'PSA', 'n_rot_bonds', 'n_rings', 'n_ar_rings',
         'n_HBA', 'n_HBD', 'Fsp3', 'logP', 'NHOHCount', 'NOCount', 'NumHeteroatoms',
        'n_positive', '_n_negative', 'n_ring_asmbl', 'n_stereo']
a=calc_descriptors(model_tox_data)
descdf=pd.DataFrame(a, columns=descs)
descdf_approved=descdf.reset_index(drop=True)
descdf_approved

[23:49:06] Unusual charge on atom 16 number of radical electrons set to zero
[23:49:07] WARNING: not removing hydrogen atom without neighbors
[23:49:07] WARNING: not removing hydrogen atom without neighbors
[23:49:07] WARNING: not removing hydrogen atom without neighbors
[23:49:07] WARNING: not removing hydrogen atom without neighbors
[23:49:07] WARNING: not removing hydrogen atom without neighbors
[23:49:07] WARNING: not removing hydrogen atom without neighbors


,PSA,n_rot_bonds,n_rings,n_ar_rings,n_HBA,n_HBD,Fsp3,logP,NHOHCount,NOCount,NumHeteroatoms,n_positive,_n_negative,n_ring_asmbl,n_stereo
0,46.32,3.0,2.0,2.0,5.0,0.0,0.500000,1.27892,0.0,5.0,5.0,22.0,8.0,0.0,0.0
1,58.64,6.0,1.0,1.0,4.0,1.0,0.428571,1.89220,1.0,5.0,5.0,26.0,13.0,0.0,0.0
2,48.39,6.0,3.0,3.0,4.0,2.0,0.250000,5.17920,2.0,4.0,5.0,31.0,16.0,0.0,0.0
3,28.16,8.0,2.0,2.0,3.0,1.0,0.500000,4.81060,1.0,3.0,4.0,32.0,16.0,0.0,2.0
4,35.91,6.0,3.0,2.0,3.0,0.0,0.333333,4.00490,0.0,4.0,6.0,33.0,17.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,76.72,5.0,3.0,3.0,6.0,1.0,0.250000,2.17690,1.0,6.0,9.0,27.0,12.0,0.0,2.0
19907,109.11,3.0,5.0,1.0,8.0,0.0,0.565217,2.53640,0.0,8.0,8.0,43.0,12.0,2.0,5.0
19908,146.35,10.0,3.0,1.0,7.0,4.0,0.583333,1.09980,5.0,10.0,10.0,46.0,23.0,0.0,2.0
19909,144.28,10.0,7.0,1.0,10.0,3.0,0.567568,3.70440,3.0,10.0,10.0,61.0,30.0,5.0,13.0


In [25]:
tox_model_data= pd.concat([model_tox_data, Morgan_fingerprint_table, MACCSfingerprint_table, descdf_approved, Mordred_table], axis=1)
tox_model_data

,smiles_r,TOXICITY,Source_rank,Source,Data,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI,0,0,0,0,1,...,6.605298,9.602585,60.727536,205.132745,6.837758,338.0,22.0,76.0,90.0,3.500000
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,9.339525,51.036465,264.147393,6.773010,844.0,23.0,86.0,93.0,4.500000
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,10.080168,59.651481,355.145140,7.556280,1580.0,39.0,128.0,149.0,5.666667
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI,0,1,0,0,0,...,0.000000,9.773777,55.480289,319.181526,6.649615,1212.0,31.0,106.0,120.0,5.138889
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI,0,0,0,0,0,...,5.379897,10.168118,70.135184,387.151368,7.743027,1747.0,48.0,138.0,163.0,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,1,0,0,0,...,6.259581,10.241209,73.142299,349.115045,8.951668,1359.0,41.0,132.0,157.0,5.458333
19907,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,0,0,0,0,...,7.669962,10.959036,84.585213,428.147118,7.784493,2102.0,69.0,184.0,237.0,6.513889
19908,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,1,1,0,0,...,0.000000,10.360058,70.097297,473.263819,6.858896,4012.0,50.0,168.0,194.0,7.833333
19909,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,0,0,0,0,...,8.945724,11.424389,108.553825,648.293448,7.124104,6986.0,109.0,278.0,362.0,10.138889


In [26]:
test_data_features= tox_model_data[tox_model_data['smiles_r'].isin(outer_test_data_tox['smiles_r'])].reset_index(drop=True)

In [27]:
train_data_features= tox_model_data[tox_model_data['smiles_r'].isin(train_data_tox['smiles_r'])].reset_index(drop=True)

In [28]:
test_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI,0,0,0,0,1,...,6.605298,9.602585,60.727536,205.132745,6.837758,338.0,22.0,76.0,90.0,3.500000
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,9.339525,51.036465,264.147393,6.773010,844.0,23.0,86.0,93.0,4.500000
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,10.080168,59.651481,355.145140,7.556280,1580.0,39.0,128.0,149.0,5.666667
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI,0,1,0,0,0,...,0.000000,9.773777,55.480289,319.181526,6.649615,1212.0,31.0,106.0,120.0,5.138889
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI,0,0,0,0,0,...,5.379897,10.168118,70.135184,387.151368,7.743027,1747.0,48.0,138.0,163.0,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,O=C(N[C@@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,1,0,0,0,...,0.000000,9.625162,52.858564,322.012327,10.062885,880.0,29.0,94.0,106.0,4.611111
1271,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,0,0,0,0,...,7.661527,11.357733,111.062995,780.429607,6.558232,14940.0,113.0,320.0,399.0,11.368056
1272,C=C1/C(=C\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,1,0,0,0,...,7.065613,10.448280,80.333949,416.329045,5.626068,2677.0,49.0,160.0,188.0,6.361111
1273,CCC(=O)NCC[C@@H]1CCc2ccc3c(c21)CCO3,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,0,0,0,0,...,7.423568,9.845488,68.245058,259.157229,6.478931,735.0,27.0,100.0,119.0,4.277778


In [29]:
train_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,C[C@H](N)C(=O)N[C@@H](C)C(=O)NC1[C@@H]2CN(c3nc...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,0,1,0,0,0,...,8.355850,10.869311,98.746533,558.183853,8.587444,5797.0,69.0,222.0,271.0,8.472222
1,CN(C)CCCC1c2ccccc2Nc2ccc(Cl)cc21,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,0,0,0,0,0,...,0.000000,10.024244,55.050898,300.139326,7.146174,896.0,33.0,110.0,129.0,4.611111
2,NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,0,0,0,0,0,...,6.555357,9.844533,73.704745,475.975849,11.069206,2264.0,32.0,126.0,139.0,5.791667
3,O=c1[nH]c(=O)n([C@@H]2O[C@H](CO)[C@@H](O)[C@@H...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,0,0,0,0,0,...,7.002156,9.881191,65.574603,371.961848,13.284352,573.0,30.0,94.0,113.0,4.000000
4,CN1CCCC(CN2c3ccccc3Sc3ccccc32)C1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,0,0,0,0,0,...,0.000000,10.137017,56.550528,310.150370,7.048872,982.0,36.0,120.0,143.0,4.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18631,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,1,0,0,0,...,6.259581,10.241209,73.142299,349.115045,8.951668,1359.0,41.0,132.0,157.0,5.458333
18632,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,0,0,0,0,...,7.669962,10.959036,84.585213,428.147118,7.784493,2102.0,69.0,184.0,237.0,6.513889
18633,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,1,1,0,0,...,0.000000,10.360058,70.097297,473.263819,6.858896,4012.0,50.0,168.0,194.0,7.833333
18634,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0,16,Reactive Metabolite Mazzolari et al,LivTox,0,0,0,0,0,...,8.945724,11.424389,108.553825,648.293448,7.124104,6986.0,109.0,278.0,362.0,10.138889


In [30]:
test_data_features.to_csv("Test_data_DILIst.csv", index=False)

In [31]:
train_data_features.to_csv("Train_data_liv_tox.csv", index=False)

In [32]:
test_data_features= pd.read_csv("Test_data_DILIst.csv")
test_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI,0,0,0,0,1,...,6.605298,9.602585,60.727536,205.132745,6.837758,338.0,22.0,76.0,90.0,3.500000
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,9.339525,51.036465,264.147393,6.773010,844.0,23.0,86.0,93.0,4.500000
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI,0,0,0,0,0,...,0.000000,10.080168,59.651481,355.145140,7.556280,1580.0,39.0,128.0,149.0,5.666667
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI,0,1,0,0,0,...,0.000000,9.773777,55.480289,319.181526,6.649615,1212.0,31.0,106.0,120.0,5.138889
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI,0,0,0,0,0,...,5.379897,10.168118,70.135184,387.151368,7.743027,1747.0,48.0,138.0,163.0,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,O=C(N[C@@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,1,0,0,0,...,0.000000,9.625162,52.858564,322.012327,10.062885,880.0,29.0,94.0,106.0,4.611111
1271,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,0,0,0,0,...,7.661527,11.357733,111.062995,780.429607,6.558232,14940.0,113.0,320.0,399.0,11.368056
1272,C=C1/C(=C\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H...,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,1,0,0,0,...,7.065613,10.448280,80.333949,416.329045,5.626068,2677.0,49.0,160.0,188.0,6.361111
1273,CCC(=O)NCC[C@@H]1CCc2ccc3c(c21)CCO3,0,13,DILIrank dataset CP Overlap REpsilber,DILI,0,0,0,0,0,...,7.423568,9.845488,68.245058,259.157229,6.478931,735.0,27.0,100.0,119.0,4.277778


In [33]:
for i in test_data_features.Source_rank.unique():
    
    print(i)
    print(test_data_features[test_data_features["Source_rank"]==i]["TOXICITY"].value_counts())
    print(len(test_data_features[test_data_features["Source_rank"]==i]))

1
1    698
0    425
Name: TOXICITY, dtype: int64
1123
13
1    122
0     30
Name: TOXICITY, dtype: int64
152
